# 🎯 Anthropic SDK - Specialty Level

## Advanced Specialized Topics

This notebook covers specialized and cutting-edge features of the Anthropic SDK for production applications.

### What You'll Learn:
- Extended Thinking (Chain of Thought reasoning)
- Batch Processing for high-volume operations
- PDF Document Processing
- Citations and Source Attribution
- Building AI Agents with complex workflows
- Production patterns and best practices

---

## 1. Setup

In [ ]:
import anthropic
import json
import base64
import httpx
import asyncio
import time
from config import MODEL, ANTHROPIC_API_KEY, validate_api_key, DEFAULT_MAX_TOKENS

validate_api_key()
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
async_client = anthropic.AsyncAnthropic(api_key=ANTHROPIC_API_KEY)

print(f"Model: {MODEL}")
print("Ready for specialty topics!")

# Note: Some features require specific models
# Extended Thinking requires Claude 3.5 Sonnet or Claude 3 Opus
THINKING_MODEL = "claude-sonnet-4-20250514"  # For extended thinking examples

## 2. Extended Thinking (Chain of Thought)

Extended Thinking allows Claude to "think" through complex problems step-by-step before providing an answer. This is powerful for:
- Complex reasoning tasks
- Math problems
- Code analysis
- Multi-step problem solving

**Note:** Extended thinking requires Claude 3.5 Sonnet or Claude 3 Opus models.

In [ ]:
# Extended Thinking Example
# This feature allows Claude to reason through complex problems

def extended_thinking_example(question, budget_tokens=5000):
    """
    Use extended thinking for complex reasoning tasks.
    
    Args:
        question: The complex question to analyze
        budget_tokens: Maximum tokens for thinking (1024-100000)
    """
    try:
        response = client.messages.create(
            model=THINKING_MODEL,
            max_tokens=16000,
            thinking={
                "type": "enabled",
                "budget_tokens": budget_tokens  # Tokens allocated for thinking
            },
            messages=[
                {"role": "user", "content": question}
            ]
        )
        
        # Process the response - it contains both thinking and text blocks
        thinking_content = ""
        response_text = ""
        
        for block in response.content:
            if block.type == "thinking":
                thinking_content = block.thinking
            elif block.type == "text":
                response_text = block.text
        
        return {
            "thinking": thinking_content,
            "response": response_text,
            "usage": {
                "input_tokens": response.usage.input_tokens,
                "output_tokens": response.usage.output_tokens
            }
        }
    except Exception as e:
        return {"error": str(e)}

# Test with a complex reasoning problem
result = extended_thinking_example(
    "A farmer has 17 sheep. All but 9 run away. How many sheep does the farmer have left? Explain your reasoning step by step."
)

if "error" not in result:
    print("🧠 Claude's Thinking Process:")
    print("=" * 60)
    print(result["thinking"][:1000] + "..." if len(result.get("thinking", "")) > 1000 else result.get("thinking", ""))
    print("\n" + "=" * 60)
    print("\n📝 Final Answer:")
    print(result["response"])
else:
    print(f"Note: Extended thinking requires a supported model. Error: {result['error']}")

In [ ]:
# Streaming with Extended Thinking
# You can also stream responses with thinking enabled

async def stream_with_thinking(question, budget_tokens=5000):
    """Stream a response with extended thinking."""
    print("🧠 Streaming thinking and response...\n")
    
    thinking_text = []
    response_text = []
    current_block_type = None
    
    try:
        async with async_client.messages.stream(
            model=THINKING_MODEL,
            max_tokens=8000,
            thinking={
                "type": "enabled",
                "budget_tokens": budget_tokens
            },
            messages=[{"role": "user", "content": question}]
        ) as stream:
            async for event in stream:
                # Handle different event types
                if hasattr(event, 'type'):
                    if event.type == 'content_block_start':
                        if hasattr(event, 'content_block'):
                            current_block_type = event.content_block.type
                            if current_block_type == "thinking":
                                print("💭 Thinking:", end=" ")
                            elif current_block_type == "text":
                                print("\n\n📝 Response:", end=" ")
                    
                    elif event.type == 'content_block_delta':
                        if hasattr(event, 'delta'):
                            if hasattr(event.delta, 'thinking'):
                                print(event.delta.thinking, end="", flush=True)
                                thinking_text.append(event.delta.thinking)
                            elif hasattr(event.delta, 'text'):
                                print(event.delta.text, end="", flush=True)
                                response_text.append(event.delta.text)
        
        print("\n\n✅ Stream complete!")
        return {
            "thinking": "".join(thinking_text),
            "response": "".join(response_text)
        }
    except Exception as e:
        print(f"\nNote: Extended thinking streaming requires a supported model. Error: {e}")
        return {"error": str(e)}

# Run the streaming example
result = await stream_with_thinking(
    "What is 15% of 240? Show your calculation."
)

## 3. Batch Processing

Batch processing allows you to send multiple requests at once for high-volume operations. This is ideal for:
- Processing large datasets
- Bulk content generation
- Cost optimization (batch requests are typically cheaper)
- Background processing jobs

In [ ]:
# Batch Processing Example
# Create multiple requests to be processed as a batch

def create_batch_requests(prompts):
    """
    Create batch requests from a list of prompts.
    
    Args:
        prompts: List of (custom_id, prompt) tuples
    
    Returns:
        List of batch request objects
    """
    requests = []
    for custom_id, prompt in prompts:
        requests.append({
            "custom_id": custom_id,
            "params": {
                "model": MODEL,
                "max_tokens": 256,
                "messages": [
                    {"role": "user", "content": prompt}
                ]
            }
        })
    return requests

# Example prompts for batch processing
prompts = [
    ("summary-1", "Summarize the concept of machine learning in one sentence."),
    ("summary-2", "Summarize the concept of neural networks in one sentence."),
    ("summary-3", "Summarize the concept of deep learning in one sentence."),
    ("translate-1", "Translate 'Hello, how are you?' to Spanish."),
    ("translate-2", "Translate 'Hello, how are you?' to French."),
]

batch_requests = create_batch_requests(prompts)
print(f"Created {len(batch_requests)} batch requests")
print("\nExample request structure:")
print(json.dumps(batch_requests[0], indent=2))

In [ ]:
# Submit and monitor a batch job
# Note: Batch processing is asynchronous - results are retrieved later

def submit_batch(requests):
    """Submit a batch of requests for processing."""
    try:
        # Create the batch
        batch = client.messages.batches.create(requests=requests)
        
        print(f"Batch created!")
        print(f"  ID: {batch.id}")
        print(f"  Status: {batch.processing_status}")
        print(f"  Created: {batch.created_at}")
        
        return batch
    except Exception as e:
        print(f"Error creating batch: {e}")
        return None

def check_batch_status(batch_id):
    """Check the status of a batch job."""
    try:
        batch = client.messages.batches.retrieve(batch_id)
        
        print(f"Batch Status: {batch.processing_status}")
        print(f"  Request counts:")
        print(f"    Processing: {batch.request_counts.processing}")
        print(f"    Succeeded: {batch.request_counts.succeeded}")
        print(f"    Errored: {batch.request_counts.errored}")
        print(f"    Canceled: {batch.request_counts.canceled}")
        print(f"    Expired: {batch.request_counts.expired}")
        
        return batch
    except Exception as e:
        print(f"Error checking batch: {e}")
        return None

def get_batch_results(batch_id):
    """Retrieve results from a completed batch."""
    try:
        results = []
        for result in client.messages.batches.results(batch_id):
            results.append({
                "custom_id": result.custom_id,
                "result_type": result.result.type,
                "response": result.result.message.content[0].text if result.result.type == "succeeded" else None,
                "error": result.result.error if result.result.type == "errored" else None
            })
        return results
    except Exception as e:
        print(f"Error getting results: {e}")
        return []

# Submit the batch (uncomment to run)
# batch = submit_batch(batch_requests)
# if batch:
#     # Wait and check status
#     time.sleep(5)
#     check_batch_status(batch.id)

print("Batch functions defined. Uncomment the code above to submit a real batch.")

## 4. PDF Document Processing

Claude can analyze PDF documents directly. This is powerful for:
- Document summarization
- Information extraction
- Question answering over documents
- Contract analysis

In [ ]:
# PDF Document Processing
# Claude can analyze PDF files sent as base64-encoded content

def analyze_pdf_from_url(pdf_url, question):
    """
    Analyze a PDF document from a URL.
    
    Args:
        pdf_url: URL to the PDF file
        question: Question to ask about the document
    
    Returns:
        Claude's analysis of the document
    """
    try:
        # Download and encode the PDF
        pdf_data = base64.standard_b64encode(httpx.get(pdf_url).content).decode("utf-8")
        
        response = client.messages.create(
            model=MODEL,
            max_tokens=2048,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "document",
                            "source": {
                                "type": "base64",
                                "media_type": "application/pdf",
                                "data": pdf_data,
                            },
                        },
                        {
                            "type": "text",
                            "text": question
                        }
                    ],
                }
            ],
        )
        
        return response.content[0].text
    except Exception as e:
        return f"Error analyzing PDF: {e}"

def analyze_pdf_from_file(file_path, question):
    """
    Analyze a PDF document from a local file.
    
    Args:
        file_path: Path to the local PDF file
        question: Question to ask about the document
    
    Returns:
        Claude's analysis of the document
    """
    try:
        with open(file_path, "rb") as f:
            pdf_data = base64.standard_b64encode(f.read()).decode("utf-8")
        
        response = client.messages.create(
            model=MODEL,
            max_tokens=2048,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "document",
                            "source": {
                                "type": "base64",
                                "media_type": "application/pdf",
                                "data": pdf_data,
                            },
                        },
                        {
                            "type": "text",
                            "text": question
                        }
                    ],
                }
            ],
        )
        
        return response.content[0].text
    except FileNotFoundError:
        return f"File not found: {file_path}"
    except Exception as e:
        return f"Error analyzing PDF: {e}"

print("PDF analysis functions defined!")
print("\nExample usage:")
print('  result = analyze_pdf_from_url("https://example.com/document.pdf", "Summarize this document")')
print('  result = analyze_pdf_from_file("./document.pdf", "What are the key points?")')

## 5. Citations and Source Attribution

Citations allow Claude to reference specific parts of provided documents, enabling:
- Verifiable responses
- Source tracking
- Academic-style references
- Fact-checking capabilities

In [ ]:
# Citations Example
# Enable citations to get references to source documents

def query_with_citations(documents, question):
    """
    Query documents with citations enabled.
    
    Args:
        documents: List of document dicts with 'title', 'content' keys
        question: Question to ask about the documents
    
    Returns:
        Response with citations
    """
    # Build content with document sources
    content = []
    
    for i, doc in enumerate(documents):
        content.append({
            "type": "document",
            "source": {
                "type": "text",
                "media_type": "text/plain",
                "data": doc["content"]
            },
            "title": doc.get("title", f"Document {i+1}"),
            "citations": {"enabled": True}  # Enable citations for this document
        })
    
    content.append({
        "type": "text",
        "text": question
    })
    
    try:
        response = client.messages.create(
            model=MODEL,
            max_tokens=1024,
            messages=[
                {
                    "role": "user",
                    "content": content
                }
            ]
        )
        
        # Process response with citations
        result = {
            "text": "",
            "citations": []
        }
        
        for block in response.content:
            if block.type == "text":
                result["text"] += block.text
                # Check for citations in the block
                if hasattr(block, 'citations') and block.citations:
                    for citation in block.citations:
                        result["citations"].append({
                            "document_title": citation.document_title,
                            "cited_text": citation.cited_text,
                            "start_index": citation.start_char_index,
                            "end_index": citation.end_char_index
                        })
        
        return result
    except Exception as e:
        return {"error": str(e), "text": "", "citations": []}

# Example documents
sample_documents = [
    {
        "title": "Python Overview",
        "content": """Python is a high-level, interpreted programming language created by Guido van Rossum 
        and first released in 1991. Python's design philosophy emphasizes code readability with the use 
        of significant indentation. Python is dynamically typed and garbage-collected. It supports 
        multiple programming paradigms, including structured, object-oriented, and functional programming."""
    },
    {
        "title": "Python Applications",
        "content": """Python is widely used in web development, data science, artificial intelligence, 
        machine learning, automation, and scientific computing. Popular frameworks include Django and 
        Flask for web development, NumPy and Pandas for data analysis, and TensorFlow and PyTorch 
        for machine learning."""
    }
]

# Query with citations
result = query_with_citations(
    sample_documents,
    "When was Python created and what are its main applications?"
)

print("Response:")
print("-" * 50)
print(result["text"])
print("\nCitations:")
print("-" * 50)
if result["citations"]:
    for i, citation in enumerate(result["citations"], 1):
        print(f"{i}. From '{citation['document_title']}':")
        print(f"   \"{citation['cited_text'][:100]}...\"" if len(citation.get('cited_text', '')) > 100 else f"   \"{citation.get('cited_text', '')}\"")
else:
    print("No citations found (citations may require specific model support)")

## 6. Building AI Agents

AI Agents are autonomous systems that can:
- Make decisions based on context
- Use tools to accomplish tasks
- Maintain state across interactions
- Handle complex multi-step workflows

In [ ]:
# AI Agent Implementation
# A complete agent that can use tools and maintain conversation state

import math

class AIAgent:
    """
    An AI Agent that can use tools to accomplish tasks.
    """
    
    def __init__(self, client, model=MODEL, system_prompt=None):
        self.client = client
        self.model = model
        self.system_prompt = system_prompt or "You are a helpful AI assistant with access to tools."
        self.conversation_history = []
        self.tools = []
        self.tool_functions = {}
        
    def register_tool(self, name, description, parameters, function):
        """Register a tool that the agent can use."""
        self.tools.append({
            "name": name,
            "description": description,
            "input_schema": parameters
        })
        self.tool_functions[name] = function
        
    def _execute_tool(self, tool_name, tool_input):
        """Execute a registered tool."""
        if tool_name in self.tool_functions:
            try:
                return self.tool_functions[tool_name](**tool_input)
            except Exception as e:
                return {"error": str(e)}
        return {"error": f"Unknown tool: {tool_name}"}
    
    def run(self, user_message, max_iterations=10):
        """
        Run the agent with a user message.
        Handles tool use automatically until completion.
        """
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        iteration = 0
        while iteration < max_iterations:
            iteration += 1
            
            # Make API call
            response = self.client.messages.create(
                model=self.model,
                max_tokens=2048,
                system=self.system_prompt,
                tools=self.tools if self.tools else None,
                messages=self.conversation_history
            )
            
            # Check if we need to use tools
            if response.stop_reason == "tool_use":
                # Process tool calls
                tool_results = []
                for block in response.content:
                    if block.type == "tool_use":
                        print(f"  🔧 Using tool: {block.name}")
                        result = self._execute_tool(block.name, block.input)
                        print(f"     Result: {json.dumps(result)[:100]}...")
                        tool_results.append({
                            "type": "tool_result",
                            "tool_use_id": block.id,
                            "content": json.dumps(result)
                        })
                
                # Add assistant response and tool results
                self.conversation_history.append({
                    "role": "assistant",
                    "content": response.content
                })
                self.conversation_history.append({
                    "role": "user",
                    "content": tool_results
                })
            else:
                # Done - extract final response
                final_response = ""
                for block in response.content:
                    if hasattr(block, "text"):
                        final_response += block.text
                
                self.conversation_history.append({
                    "role": "assistant",
                    "content": final_response
                })
                
                return final_response
        
        return "Max iterations reached without completion."
    
    def reset(self):
        """Reset conversation history."""
        self.conversation_history = []

print("AIAgent class defined!")

In [ ]:
# Create and use an AI Agent with tools

# Create agent
agent = AIAgent(
    client,
    system_prompt="You are a helpful research assistant. Use tools when needed to answer questions accurately."
)

# Register calculator tool
agent.register_tool(
    name="calculator",
    description="Perform mathematical calculations",
    parameters={
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "Math expression to evaluate (e.g., '2 + 2', 'sqrt(16)')"
            }
        },
        "required": ["expression"]
    },
    function=lambda expression: {
        "result": eval(expression, {"__builtins__": {}, "sqrt": math.sqrt, "pow": pow, "abs": abs})
    }
)

# Register knowledge base tool
knowledge_base = {
    "python": "Python is a programming language created in 1991 by Guido van Rossum.",
    "javascript": "JavaScript is a programming language created in 1995 by Brendan Eich.",
    "anthropic": "Anthropic is an AI safety company founded in 2021, creator of Claude."
}

agent.register_tool(
    name="lookup",
    description="Look up information about a topic",
    parameters={
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "Topic to look up (e.g., 'python', 'javascript', 'anthropic')"
            }
        },
        "required": ["topic"]
    },
    function=lambda topic: {
        "info": knowledge_base.get(topic.lower(), f"No information found for '{topic}'")
    }
)

print("Agent created with 2 tools: calculator, lookup")
print("\nRunning agent...")
print("=" * 60)

result = agent.run("What is Python and what is 25 * 4 + 10?")

print("=" * 60)
print("\nFinal Response:")
print(result)

## 7. Production Best Practices

Key patterns for production deployments:

In [ ]:
# Production-Ready Client Wrapper
import random
from typing import Optional, List, Dict, Any
from dataclasses import dataclass
from datetime import datetime

@dataclass
class APICallMetrics:
    """Metrics for API calls."""
    timestamp: datetime
    model: str
    input_tokens: int
    output_tokens: int
    latency_ms: float
    success: bool
    error: Optional[str] = None

class ProductionClient:
    """
    Production-ready Anthropic client with:
    - Automatic retries with exponential backoff
    - Request/response logging
    - Metrics collection
    - Rate limiting
    - Circuit breaker pattern
    """
    
    def __init__(self, client, max_retries=3, base_delay=1.0):
        self.client = client
        self.max_retries = max_retries
        self.base_delay = base_delay
        self.metrics: List[APICallMetrics] = []
        self.consecutive_failures = 0
        self.circuit_open = False
        self.circuit_open_until = None
        
    def _should_retry(self, exception) -> bool:
        """Determine if we should retry based on exception type."""
        retryable_errors = (
            anthropic.RateLimitError,
            anthropic.APIConnectionError,
            anthropic.InternalServerError,
        )
        return isinstance(exception, retryable_errors)
    
    def _calculate_delay(self, attempt: int) -> float:
        """Calculate delay with exponential backoff and jitter."""
        delay = self.base_delay * (2 ** attempt)
        jitter = random.uniform(0, 0.1 * delay)
        return min(delay + jitter, 60)  # Cap at 60 seconds
    
    def _check_circuit(self) -> bool:
        """Check if circuit breaker allows requests."""
        if not self.circuit_open:
            return True
        if datetime.now() > self.circuit_open_until:
            self.circuit_open = False
            self.consecutive_failures = 0
            return True
        return False
    
    def _record_failure(self):
        """Record a failure for circuit breaker."""
        self.consecutive_failures += 1
        if self.consecutive_failures >= 5:
            self.circuit_open = True
            from datetime import timedelta
            self.circuit_open_until = datetime.now() + timedelta(seconds=30)
    
    def create_message(self, **kwargs) -> Any:
        """Create a message with production safeguards."""
        if not self._check_circuit():
            raise Exception("Circuit breaker is open - too many recent failures")
        
        start_time = time.time()
        last_exception = None
        
        for attempt in range(self.max_retries):
            try:
                response = self.client.messages.create(**kwargs)
                
                # Record success metrics
                latency = (time.time() - start_time) * 1000
                self.metrics.append(APICallMetrics(
                    timestamp=datetime.now(),
                    model=kwargs.get("model", "unknown"),
                    input_tokens=response.usage.input_tokens,
                    output_tokens=response.usage.output_tokens,
                    latency_ms=latency,
                    success=True
                ))
                
                self.consecutive_failures = 0
                return response
                
            except Exception as e:
                last_exception = e
                
                if self._should_retry(e) and attempt < self.max_retries - 1:
                    delay = self._calculate_delay(attempt)
                    print(f"Retrying in {delay:.2f}s (attempt {attempt + 1}/{self.max_retries})")
                    time.sleep(delay)
                else:
                    break
        
        # Record failure
        latency = (time.time() - start_time) * 1000
        self.metrics.append(APICallMetrics(
            timestamp=datetime.now(),
            model=kwargs.get("model", "unknown"),
            input_tokens=0,
            output_tokens=0,
            latency_ms=latency,
            success=False,
            error=str(last_exception)
        ))
        
        self._record_failure()
        raise last_exception
    
    def get_metrics_summary(self) -> Dict:
        """Get a summary of collected metrics."""
        if not self.metrics:
            return {"message": "No metrics collected yet"}
        
        successful = [m for m in self.metrics if m.success]
        failed = [m for m in self.metrics if not m.success]
        
        return {
            "total_calls": len(self.metrics),
            "successful": len(successful),
            "failed": len(failed),
            "success_rate": len(successful) / len(self.metrics) * 100,
            "total_input_tokens": sum(m.input_tokens for m in successful),
            "total_output_tokens": sum(m.output_tokens for m in successful),
            "avg_latency_ms": sum(m.latency_ms for m in successful) / len(successful) if successful else 0,
            "circuit_breaker_open": self.circuit_open
        }

# Create production client
prod_client = ProductionClient(client)

# Test it
response = prod_client.create_message(
    model=MODEL,
    max_tokens=100,
    messages=[{"role": "user", "content": "Hello!"}]
)

print("Response:", response.content[0].text)
print("\nMetrics Summary:")
print(json.dumps(prod_client.get_metrics_summary(), indent=2))

## 8. Practice Exercises

In [ ]:
# Exercise 1: Create an agent with a custom tool
# Add a "get_time" tool that returns the current time
# Your code here:


In [ ]:
# Exercise 2: Implement a batch processor for sentiment analysis
# Process 10 product reviews and classify them as positive/negative
# Your code here:


In [ ]:
# Exercise 3: Extend ProductionClient with request logging to a file
# Your code here:


## Summary

In this specialty notebook, you learned:

1. **Extended Thinking** - Chain of thought reasoning for complex problems
2. **Batch Processing** - High-volume operations with cost optimization
3. **PDF Processing** - Document analysis and information extraction
4. **Citations** - Source attribution for verifiable responses
5. **AI Agents** - Building autonomous systems with tool use
6. **Production Patterns** - Retries, circuit breakers, and metrics collection

---

## Learning Path Complete!

Congratulations! You've completed the Anthropic SDK learning path:

| Level | Topics Covered |
|-------|---------------|
| **Entry** | Client setup, basic API calls, messages, system prompts |
| **Middle** | Streaming, temperature, vision, token counting |
| **Advanced** | Tool use, JSON output, prompt caching, async operations |
| **Specialty** | Extended thinking, batches, PDFs, citations, agents |

### Next Steps:
- Build a real application using these concepts
- Explore the [Anthropic Documentation](https://docs.anthropic.com/)
- Join the [Anthropic Discord](https://discord.gg/anthropic) community
- Check out the [Anthropic Cookbook](https://github.com/anthropics/anthropic-cookbook) for more examples